# Core Functions
> Essential functions for processing and visualizing underwater cave SONAR and vision data.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:

#| export

import math
import numpy as np

def f_q2roll_pitch_yaw(v_quaternion):
    """
    Convert a quaternion to roll, pitch, and yaw angles (Euler angles).

    This implementation assumes normalized quaternion converts to Euler angles in a 3-2-1 sequence.
    The 3-2-1 or YZX convention is commonly used for Tait-Bryan angles or nautical angles.
    See wiki https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles
    for more details

    Parameters:
        v_quaternion (np.ndarray): A 4-element array representing the quaternion in the form (qx, qy, qz, qw).

    Returns:
        tuple: A tuple containing the roll, pitch, and yaw angles in radians.

    Notes:
        - The function handles singularities at the North and South Poles.
        - The conversion uses the algorithm described in the referenced wiki page.
        - Quaternions should be normalized for accurate results.

    References:
        1. [Conversion between quaternions and Euler angles](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)
        2. [Handling singularities](https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToEuler/)

    Examples:
        >>> v_quat = np.array([0.1, 0.2, 0.3, 0.4])
        >>> d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat)
        >>> print(f"Roll: {d_roll:.4f}, Pitch: {d_pitch:.4f}, Yaw: {d_yaw:.4f}")
        Roll: 0.5760, Pitch: 0.5899, Yaw: 1.3770
    """
    d_qx, d_qy, d_qz, d_qw = v_quaternion

    # Manage singularity and potential rounding error that create a not normalized quaternion
    # More information at https://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToEuler/
    d_test = d_qz * d_qw + d_qx * d_qy
    
    d_sqw = d_qw * d_qw
    d_sqx = d_qx * d_qx
    d_sqy = d_qy * d_qy
    d_sqz = d_qz * d_qz
    d_unit = d_sqx + d_sqy + d_sqz + d_sqw  # Correction factor if not normalized
    
    # Singularity at North Pole
    if d_test > 0.499 * d_unit:
        d_yaw = 2 * math.atan2(d_qx, d_qw)
        d_pitch = np.pi / 2
        d_roll = 0
        return d_roll, d_pitch, d_yaw

    # Singularity at South Pole
    if d_test < -0.499 * d_unit:
        d_yaw = -2 * math.atan2(d_qx, d_qw)
        d_pitch = -np.pi / 2
        d_roll = 0
        return d_roll, d_pitch, d_yaw

    # Roll (x-axis rotation)
    d_roll = math.atan2(2.0 * (d_qw * d_qx + d_qy * d_qz), 1.0 - 2.0 * (d_sqx + d_sqy))
    
    # Pitch (y-axis rotation)
    # CORRECTION from the docs here, math.asin(2*test/unit) is wrong (in our case)
    # pitch = 2*math.atan2(1.0 + 2.0 * (qw * qy - qx * qz),1.0 - 2.0 * (qw * qy - qx * qz)) - np.pi/2
    d_pitch = math.asin(2 * (d_qy * d_qw - d_qz * d_qx) / d_unit)
    
    # Yaw (z-axis rotation)
    d_yaw = math.atan2(2.0 * (d_qw * d_qz + d_qx * d_qy), 1.0 - 2.0 * (d_sqy + d_sqz))
    
    return d_roll, d_pitch, d_yaw

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# | hide
from nbdev.showdoc import *
import pytest

# | export
def test_f_q2roll_pitch_yaw():
    """Test the quaternion to roll, pitch, yaw conversion function."""
    # Test identity quaternion
    v_quat_identity = np.array([0, 0, 0, 1])
    d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat_identity)
    assert np.allclose([d_roll, d_pitch, d_yaw], [0, 0, 0], atol=1e-6), "Identity quaternion test failed"

    # Test 90-degree rotation around x-axis
    v_quat_x90 = np.array([np.sqrt(2)/2, 0, 0, np.sqrt(2)/2])
    d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat_x90)
    assert np.allclose([d_roll, d_pitch, d_yaw], [np.pi/2, 0, 0], atol=1e-6), "90-degree x-rotation test failed"

    # Test North Pole singularity
    v_quat_north = np.array([0, np.sqrt(2)/2, 0, np.sqrt(2)/2])
    d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat_north)
    assert np.isclose(d_pitch, np.pi/2, atol=1e-6), "North Pole singularity test failed"

    # Test South Pole singularity
    v_quat_south = np.array([0, -np.sqrt(2)/2, 0, np.sqrt(2)/2])
    d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat_south)
    assert np.isclose(d_pitch, -np.pi/2, atol=1e-6), "South Pole singularity test failed"

    print("All tests passed successfully!")

# Example usage
v_quat_example = np.array([0.1, 0.2, 0.3, 0.4])
d_roll, d_pitch, d_yaw = f_q2roll_pitch_yaw(v_quat_example)
print(f"Roll: {d_roll:.4f}, Pitch: {d_pitch:.4f}, Yaw: {d_yaw:.4f}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()